In [2]:
import os

from photutils.utils import ImageDepth
from photutils.segmentation import SourceFinder

from astropy.io import fits

In [ ]:
def measure():

    home = os.getcwd()
    data = f'{home}/data/'
    
    f275w_file = f'{data}/V5.0_PSZ1G311.65-18.48_F275W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits'

    f275w_header = fits.getheader(f275w_file)

    inverse_sensitivity = f275w_header['PHOTFLAM']

    f275w = fits.getdata(f275w_file)

    f275w = f275w * inverse_sensitivity

    f275w = f275w * 2709.7**2 / (3e8 * 1e10)

    depth = ImageDepth(4., nsigma=5., mask_pad=4., napers=10, niters=1, seed=123, zeropoint=-48.6, progress_bar=False)

    